<a href="https://colab.research.google.com/github/shrutishrinivasan/CIA_DAA/blob/main/GeneticAlgo(NN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data = pd.read_csv("loan.csv")
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [2]:
df = data
# removing irrelevant columns like Id, Zip code
# and shifting the target variable to the last position
df = df.drop(['ID','ZIP Code','Personal Loan'],axis=1)
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,1,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1


In [3]:
# Target variable: Personal Loan
df['Perosnal Loan'] = data['Personal Loan']
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Perosnal Loan
0,25,1,49,4,1.6,1,0,1,0,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1,0


In [4]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=0.25)
X_train

array([[ 51.,  25., 163., ...,   0.,   1.,   0.],
       [ 42.,  17.,  44., ...,   1.,   1.,   1.],
       [ 35.,  11.,  24., ...,   0.,   0.,   0.],
       ...,
       [ 26.,   1.,  24., ...,   0.,   0.,   1.],
       [ 57.,  33.,  49., ...,   1.,   1.,   1.],
       [ 33.,   9.,  38., ...,   0.,   0.,   0.]])

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
from keras.models import Sequential
from keras.layers import Dense

In [8]:
# neural network parameters
max_node = 5
num_hiddenlayers = 2
nn_in = X_train.shape[1]
nn_hidden1 = max_node
nn_hidden2 = max_node
nn_out = 1

In [9]:
def network(nn_in,nn_hidden1,nn_hidden2,nn_out):
    model = Sequential()
    model.add(Dense(nn_hidden1,input_dim=nn_in,activation='relu'))
    model.add(Dense(nn_hidden2,activation='relu'))
    model.add(Dense(nn_out,activation='linear'))
    
    # compile model
    model.compile(loss='mse',optimizer='adam',metrics=['mae'])
    return model

In [10]:
def run_episode(X_train,y_train,X_test,y_test,nn_in,nn_out,policy):
    nn_hidden1,nn_hidden2 = policy
    
    # build model
    model = network(nn_in,nn_hidden1,nn_hidden2,nn_out)
    
    # train model
    model.fit(X_train,y_train,epochs=30,verbose=0,validation_split=0.05)
    
    # performance
    _,accuracy = model.evaluate(X_test,y_test)
    return accuracy

In [11]:
# test model1
policy = [5,2]
acc1 = run_episode(X_train,y_train,X_test,y_test,nn_in,nn_out,policy)

# test model2
policy = [5,5]
acc2 = run_episode(X_train,y_train,X_test,y_test,nn_in,nn_out,policy)

40/40 [==============================] - 0s 2ms/step - loss: 0.0273 - mae: 0.0722


In [12]:
print("Model 1:",acc1)
print("Model 2:",acc2)

Model 1: 0.07095159590244293
Model 2: 0.07219818979501724


In [13]:
# genetic algorithm
def evaluate_policy(X_train,y_train,X_test,y_test,nn_in,nn_out,policy,n_episodes=1):
    total_rewards = 0.0
    for _ in range(n_episodes):
        total_rewards += 1/run_episode(X_train,y_train,X_test,y_test,nn_in,nn_out,policy)
    return total_rewards/n_episodes

In [14]:
def gen_random_policy(max_mode,num_hiddenlayers):
    return np.random.choice(max_node+1,size=((num_hiddenlayers)))

In [15]:
def crossover(policy1,policy2,num_hiddenlayers):
    new_policy = policy1.copy()
    for i in range(num_hiddenlayers):
        rand = np.random.uniform()
        if rand > 0.5:
            new_policy[i] = policy2[i]
    return new_policy

In [19]:
def mutation(policy1,num_hiddenlayers,max_mode,p=0.05):
    new_policy = policy1.copy()
    for i in range(num_hiddenlayers):
        rand = np.random.uniform()
        if rand < p:
            new_policy[i] = np.random.choice(max_node+1)
    return new_policy

In [20]:
import random
import numpy as np
import time

if __name__ == '__main__':
    random.seed(1234)
    np.random.seed(1234)
    
    # policy search
    n_policy = 10
    n_steps = 5
    start = time.time()
    
    policy_pop = [gen_random_policy(max_node,num_hiddenlayers) for _ in range(n_policy)]
    for idx in range(n_steps):
        policy_scores = [evaluate_policy(X_train,y_train,X_test,y_test,nn_in,nn_out,p) for p in policy_pop]
        #print('Generation %d : MaxScore=%0.2f & AvgScore=%0.3f' %(idx+1,max(policy_scores),sum(policy_scores)/len(policy_scores)))
        
        # picking the best individuals
        policy_ranks = list(reversed(np.argsort(policy_scores)))
        elite_set = [policy_pop[x] for x in policy_ranks[:5]]
        select_probs = np.array(policy_scores)/np.sum(policy_scores)
        
        if np.sum(policy_scores)==0:
            pp = 1/np.array(policy_scores).size
            select_probs = pp*np.ones(np.array(policy_scores).size)
        child_set = [crossover(
            policy_pop[np.random.choice(range(n_policy),p=select_probs)],
            policy_pop[np.random.choice(range(n_policy),p=select_probs)],num_hiddenlayers)
            
            for _ in range(n_policy-5)]
        
        # mutation
        mutated_list = [mutation(p,num_hiddenlayers,max_node) for p in child_set]
        policy_pop = elite_set
        policy_pop += mutated_list
        
    policy_score = [evaluate_policy(X_train,y_train,X_test,y_test,nn_in,nn_out,p) for p in policy_pop]
    best_policy = policy_pop[np.argmax(policy_score)]
    
    end = time.time()
    #print('Best policy score:%0.2f Time taken(sec)=%4.4f Average Score= %0.3f'
     #    %(np.max(policy_score),(end-start),sum(policy_score)/np.sum(policy_score)))
    print('Best policy is:',best_policy)

40/40 [==============================] - 0s 2ms/step - loss: 0.0347 - mae: 0.0886
Best policy is: [4 5]


In [21]:
policy = [4,5]
final_acc = run_episode(X_train,y_train,X_test,y_test,nn_in,nn_out,policy)
print("Final Accuracy:",final_acc)

40/40 [==============================] - 0s 2ms/step - loss: 0.0181 - mae: 0.0651
Final Accuracy: 0.06514447182416916
